In [ ]:
import sys
from collections import deque

class Node:
    """A node for the Huffman tree."""
    def __init__(self, freq, char=None, left=None, right=None):
        self.freq = freq
        self.char = char  # Will be None for synthetic/internal nodes
        self.left = left
        self.right = right

    def __repr__(self):
        # A helper for printing, shows the character if it's a leaf
        if self.char:
            return f"Node(freq={self.freq}, char='{self.char}')"
        return f"Node(freq={self.freq})"

def extract_min(q1, q2):
    """
    Helper function to extract the node with the minimum frequency
    from the front of either queue (q1 for leaves, q2 for synthetic nodes).
    """
    # If one queue is empty, the min must be in the other
    if not q1:
        return q2.popleft()
    if not q2:
        return q1.popleft()

    # Both queues have nodes, so compare their fronts
    if q1[0].freq <= q2[0].freq:
        return q1.popleft()
    else:
        return q2.popleft()

def linear_time_huffman(char_freqs):
    """
    Constructs a Huffman tree in O(n) time.

    Args:
        char_freqs: A list of (char, freq) tuples,
                    **assumed to be sorted by frequency in ascending order.**
    """
    n = len(char_freqs)
    if n == 0:
        return None

    # 1. Initialize Q1 (for leaf characters)
    #    Use deque for efficient O(1) pops from the left
    q1 = deque()
    for char, freq in char_freqs:
        q1.append(Node(freq=freq, char=char))

    # 2. Initialize Q2 (for synthetic characters)
    q2 = deque()

    # Handle the edge case of only one character
    if n == 1:
        # Create a dummy parent node
        v1 = q1.popleft()
        root = Node(freq=v1.freq, left=v1, right=None)
        return root

    # 3. Loop n-1 times to build the tree
    for _ in range(n - 1):
        # 4a. Find first min (v1)
        v1 = extract_min(q1, q2)

        # 4b. Find second min (v2)
        v2 = extract_min(q1, q2)

        # 4c. Create new synthetic node
        parent_freq = v1.freq + v2.freq
        parent_node = Node(freq=parent_freq, left=v1, right=v2)

        # 4d. Enqueue new node onto Q2
        q2.append(parent_node)

    # 5. The last node in Q2 is the root of the tree
    return q2.popleft()

def get_huffman_codes(root):
    """
    Traverses the tree to generate the Huffman codes for each character.
    Returns a dictionary of {char: code}.
    """
    codes = {}

    def _traverse(node, current_code):
        if node is None:
            return

        # If this is a leaf node, store the code
        if node.char:
            # Handle edge case of single-node tree (code is '0' or '1')
            if not current_code:
                codes[node.char] = '0'
            else:
                codes[node.char] = current_code
            return

        # Recurse left (add '0') and right (add '1')
        _traverse(node.left, current_code + '0')
        _traverse(node.right, current_code + '1')

    _traverse(root, "")
    return codes

# --- Example from geek-for-geeks ---
if __name__ == "__main__":
    # 1. Define character frequencies from the new example
    freq_data = [('a', 5),('b', 9),('c', 12),('d', 13),('e', 16),('f', 45)]
    #freq_data = [('a', 4), ('e', 5), ('f', 2), ('h', 1), ('i', 2), ('l', 1),('m', 2), ('n', 2), ('o', 1), ('p', 1), ('r', 1), ('s', 2),('t', 2), ('u', 1), ('x', 1), (' ', 5)]

    # 2. Sort the data by frequency:
    freq_data.sort(key=lambda item: item[1])

    print(f"Sorted input data (n={len(freq_data)}):")
    for char, freq in freq_data:
        print(f"  '{char}': {freq}")

    # 3. Build the tree
    print("\nBuilding tree...")
    huffman_root = linear_time_huffman(freq_data)

    # 4. Get and print the codes
    print("\nGenerating codes...")
    codes = get_huffman_codes(huffman_root)

    print("\n--- Huffman Codes (O(n) Algorithm) ---")
    # Sort for cleaner output
    for char in sorted(codes.keys()):
        print(f"  '{char}': {codes[char]}")

    print(f"\nRoot node frequency: {huffman_root.freq}")

Sorted input data (n=6):
  'a': 5
  'b': 9
  'c': 12
  'd': 13
  'e': 16
  'f': 45

Building tree...

Generating codes...

--- Huffman Codes (O(n) Algorithm) ---
  'a': 1100
  'b': 1101
  'c': 100
  'd': 101
  'e': 111
  'f': 0

Root node frequency: 100
